<a href="https://colab.research.google.com/github/DarkoMedin/Python-AI-Agent-Intro-/blob/main/Intro_to_Agents_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU huggingface_hub

import os, getpass
from huggingface_hub import InferenceClient

os.environ["HF_TOKEN"] = getpass.getpass("HF token: ")
client = InferenceClient("moonshotai/Kimi-K2-Thinking")



In [ ]:
res = client.chat.completions.create(
    messages=[{"role": "user", "content": "Tell me about Dinosaurs."}]
)

msg = res.choices[0].message
print(msg.content)


In [ ]:
print(msg.reasoning_content)

In [ ]:
import json

def lookup_order(order_id: str) -> str:
    orders = {
        "A1001": {"status": "shipped", "eta_days": 2, "items": 3},
        "A1002": {"status": "processing", "eta_days": 5, "items": 1},
        "A1003": {"status": "delivered", "eta_days": 0, "items": 2},
    }
    oid = order_id.strip().upper()
    return json.dumps({"order_id": oid, **orders.get(oid, {"status": "not_found"})})


In [ ]:
tool = {
    "type": "function",
    "function": {
        "name": "lookup_order",
        "description": "Look up an order in the store system by order_id.",
        "parameters": {
            "type": "object",
            "properties": {"order_id": {"type": "string"}},
            "required": ["order_id"],
        },
    },
}


In [ ]:
TOOLS = {"lookup_order": lookup_order}

def agent(prompt: str, system="You are a helpful assistant. Use tools when needed."):
    messages = [{"role": "system", "content": system}, {"role": "user", "content": prompt}]

    while True:
        res = client.chat.completions.create(messages=messages, tools=[tool], tool_choice="auto")
        msg = res.choices[0].message
        messages.append(msg)

        if not msg.tool_calls:
            return msg.content

        for tc in msg.tool_calls:
            args = json.loads(tc.function.arguments or "{}")
            out = TOOLS[tc.function.name](**args)
            messages.append({
                "role": "tool",
                "tool_call_id": tc.id,
                "name": tc.function.name,
                "content": str(out),
            })


In [ ]:
print(agent("My order is A1001. Where is it and when will it arrive?"))


In [ ]:
print(agent("I want to use tools and analyse dinosaurs"))